In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git

In [ ]:
from collections import defaultdict
from glob import glob
from random import choice, sample

import cv2
import numpy as np
import pandas as pd

import keras.backend as K
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.layers import Lambda
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D
from keras.layers import Concatenate, Multiply, Dropout, Subtract, Add
from keras.models import Model
from keras import layers, models, optimizers
from keras.optimizers import Adam
from keras_vggface.utils import preprocess_input
from keras_vggface.vggface import VGGFace

from imgaug import augmenters as iaa

In [ ]:
train_file_path = "../input/train_relationships.csv"
train_folders_path = "../input/train/"
val_families = "F09"

all_images = glob(train_folders_path + "*/*/*.jpg")

train_images = [x for x in all_images if val_families not in x]
val_images = [x for x in all_images if val_families in x]

train_person_to_images_map = defaultdict(list)
val_person_to_images_map = defaultdict(list)

ppl = [x.split("/")[-3] + "/" + x.split("/")[-2] for x in all_images]

for x in train_images:
    train_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    
for x in val_images:
    val_person_to_images_map[x.split("/")[-3] + "/" + x.split("/")[-2]].append(x)
    
relationships = pd.read_csv(train_file_path)
relationships = list(zip(relationships.p1.values, relationships.p2.values))
relationships = [x for x in relationships if x[0] in ppl and x[1] in ppl]

train = [x for x in relationships if val_families not in x[0]]
val = [x for x in relationships if val_families in x[0]]

In [ ]:
def read_img(path):
    img = cv2.imread(path)
    seq = iaa.Sequential([
        iaa.OneOf([
            iaa.Sometimes(0.25,
                iaa.Dropout(p=(0, 0.08)),
            ),
            iaa.Sometimes(0.25,
                iaa.GaussianBlur(sigma=(0.0, 1.0)),
            ),
            iaa.Sometimes(0.25,
                iaa.Add((-20, 20)),
            ),
            iaa.Sometimes(0.25,
                iaa.AddElementwise((-20, 20)),
            ),
            iaa.Sometimes(0.25,
                iaa.AdditiveGaussianNoise(scale=(0, 0.01*255)),
            )
        ])
    ], random_order=True)
    img = seq.augment_image(img)
    img = np.array(img).astype(np.float)
    img = preprocess_input(img, version=2)
    return img


def gen(list_tuples, person_to_images_map, batch_size=16):
    ppl = list(person_to_images_map.keys())
    while True:
        batch_tuples = sample(list_tuples, batch_size // 2)
        labels = [1] * len(batch_tuples)
        while len(batch_tuples) < batch_size:
            p1 = choice(ppl)
            p2 = choice(ppl)

            if p1 != p2 and (p1, p2) not in list_tuples and (p2, p1) not in list_tuples:
                batch_tuples.append((p1, p2))
                labels.append(0)

        for x in batch_tuples:
            if not len(person_to_images_map[x[0]]):
                print(x[0])

        X1 = [choice(person_to_images_map[x[0]]) for x in batch_tuples]
        X1 = np.array([read_img(x) for x in X1])

        X2 = [choice(person_to_images_map[x[1]]) for x in batch_tuples]
        X2 = np.array([read_img(x) for x in X2])

        yield [X1, X2], labels

In [ ]:
# https://github.com/keras-team/keras/issues/2672#issuecomment-218177572

def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

In [ ]:
def baseline_model():
    input_1 = Input(shape=(224, 224, 3))
    input_2 = Input(shape=(224, 224, 3))

    base_model = VGGFace(model='resnet50', include_top=False)
    base_model = Model(base_model.input, base_model.layers[-2].output)

#     for x in base_model.layers[:-2]:
#         x.trainable = True

    x1 = base_model(input_1)  # (None, 7, 7, 2048)
    x2 = base_model(input_2)
    
    x1 = Concatenate(axis=-1)([GlobalAvgPool2D()(x1), GlobalAvgPool2D()(x1)])
    x2 = Concatenate(axis=-1)([GlobalAvgPool2D()(x2), GlobalAvgPool2D()(x2)])

    x3 = Subtract()([x1, x2])
    x4 = Multiply()([x3, x3])

    x5 = Multiply()([x1, x1])
    x6 = Multiply()([x2, x2])
    x7 = Subtract()([x5, x6])

    x = Concatenate(axis=-1)([x5, x6, Multiply()([x1, x2])])

    x = Dense(2048, activation="relu")(x)
    x = Dropout(0.01)(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.01)(x)
    x = Dense(128, activation="relu")(x)
    
    cos_dis = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([x1, x2])
    
    x = Concatenate(axis=-1)([x, cos_dis])
    
    x = Dense(32, activation="relu")(x)
    x = Dropout(0.1)(x)
    out = Dense(1, activation="sigmoid")(x)

    model = Model([input_1, input_2], out)

    model.compile(loss="binary_crossentropy", metrics=['acc'], optimizer=Adam(0.00001))

    model.summary()

    return model

In [ ]:
vgg = VGGFace(model='resnet50', include_top=False)
vgg.summary()

In [ ]:
file_path = "vgg_face.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
reduce_on_plateau = ReduceLROnPlateau(monitor="val_acc", mode="max", factor=0.1, patience=20, verbose=1)
callbacks_list = [checkpoint, reduce_on_plateau]

In [ ]:
model = baseline_model()

In [ ]:
gc.collect()

In [ ]:
model.fit_generator(gen(train, train_person_to_images_map, batch_size=16), 
                    use_multiprocessing=True,
                    validation_data=gen(val, val_person_to_images_map, batch_size=16), 
                    epochs=100, 
                    verbose=2,
                    workers=4, 
                    callbacks=callbacks_list, 
                    steps_per_epoch=200, 
                    validation_steps=100)

In [ ]:
# model.load_weights(file_path)

In [ ]:
test_path = "../input/test/"

def chunker(seq, size=32):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

def read_img(path):
    img = cv2.imread(path)
    img = np.array(img).astype(np.float)
    return preprocess_input(img, version=2)

In [ ]:
from tqdm import tqdm

submission = pd.read_csv('../input/sample_submission.csv')

predictions = []

for batch in tqdm(chunker(submission.img_pair.values)):
    X1 = [x.split("-")[0] for x in batch]
    X1 = np.array([read_img(test_path + x) for x in X1])

    X2 = [x.split("-")[1] for x in batch]
    X2 = np.array([read_img(test_path + x) for x in X2])

    pred = model.predict([X1, X2]).ravel().tolist()
    predictions += pred

In [ ]:
submission['is_related'] = predictions
submission.to_csv("vgg_face.csv", index=False)